In [56]:
import pandas as pd
import os
import shutil

import pandas as pd
import json
import geopandas as gpd
import os
%matplotlib inline
import geopandas as gp
import pandas as pd
import math
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping
import scipy.interpolate
import time
import folium

import networkx as nx


In [57]:
def return_start_point(x):
    #print(x)
    try:
        return x.boundary[0]
    except:
        return None

def return_end_point(x):
    try:
        return x.boundary[1]
    except:
        return None

def generate_csv(filename):
    main_arr = []
    with open('raw_kibana/' + filename + '.json') as f:
        here_traffic_flow = f.readlines()
        for line in here_traffic_flow:
            line = json.loads(line)
            line = line['_source']
            #print(line)
            for flow_item_s in line['FIS']:
                for flow_item in flow_item_s['FI']:
                    #print(flow_item)
                    temp_arr = []
                    for tmc_item in flow_item['TMC']:
                        #print(flow_item['TMC'][tmc_item])
                        temp_arr.append(flow_item['TMC'][tmc_item])

                    ## To get SHP
                    #shp_values = flow_item['SHP'][0]['value'][0]

                    arr_lon = []
                    arr_lat = []

                    fc_temp = flow_item['SHP'][0]['FC']

                    for shp in flow_item['SHP']:
                        shp_values = shp['value'][0]
                        coordinates_arr = shp_values.split(' ')
                        #print(coordinates_arr)

                        for item in coordinates_arr:
                            if len(item) > 0:
                                item = item.split(',')
                                arr_lat.append(float(item[0]))
                                arr_lon.append(float(item[1]))

                        #if fc_temp != shp['FC']:
                            #print(fc_temp, shp['FC'])
                            #print(line['PBT'])
                            #print(line['DE'])
                            #print('FC Diff')

                    polygon_geom = LineString(zip(arr_lon, arr_lat))
                    #temp_arr.append(flow_item['SHP'][0]['value'])
                    temp_arr.append(polygon_geom)

                    temp_arr.append(flow_item['SHP'][0]['FC'])


                    ## To get all SHP
                    """
                    for cf_item in flow_item['SHP'][0]:   
                        temp_arr.append(flow_item['SHP'][0][cf_item])


                    for cf_item in flow_item['CF'][0]:   
                        temp_arr.append(flow_item['CF'][0][cf_item])
                    """
                    try:
                        temp_arr.append(flow_item['CF'][0]['SP'])
                        temp_arr.append(flow_item['CF'][0]['SU'])
                        temp_arr.append(flow_item['CF'][0]['FF'])
                        temp_arr.append(flow_item['CF'][0]['JF'])
                        temp_arr.append(flow_item['CF'][0]['CN'])

                        temp_arr.append(line['PBT'])
                        temp_arr.append(line['DE'])



                        #print(len(temp_arr))
                        if len(temp_arr) == 13:
                            main_arr.append(temp_arr)
                    except:
                        pass
                    
    here_traffic_flow_df = pd.DataFrame(main_arr)
    here_traffic_flow_df.columns = ['TMC_PC', 'TMC_DE', 'TMC_QD', 'TMC_LE',
                            'Geometry', 'FC',
                             'CF_SP', 'CF_SU', 'CF_FF', 'CF_JF', 'CF_CN',
                               'PBT', 'DE']
    here_traffic_flow_gdf = gpd.GeoDataFrame(here_traffic_flow_df, geometry='Geometry')
    here_traffic_flow_gdf['endpoints'] = here_traffic_flow_gdf['Geometry'].apply(lambda x: x.boundary)
    here_traffic_flow_gdf['start_point'] = here_traffic_flow_gdf['Geometry'].apply(lambda x: return_start_point(x))
    here_traffic_flow_gdf['end_point'] = here_traffic_flow_gdf['Geometry'].apply(lambda x: return_end_point(x))

    gadm = gpd.read_file('gadm_data/gadm36_MYS_2.shp')
    kl_selangor_gadm_mys2 = gadm[gadm['NAME_1'].isin(['Kuala Lumpur', 'Selangor'])]
    from shapely.geometry import Point, Polygon

    data = []
    for index2, traffic in here_traffic_flow_gdf.iterrows():
        for index, gadm in kl_selangor_gadm_mys2.iterrows():
            if gadm['geometry'].intersects(traffic['Geometry']):
                start_district = None
                end_district = None
                try:
                    x_start = traffic['start_point'].x
                    y_start = traffic['start_point'].y

                    x_end = traffic['end_point'].x
                    y_end = traffic['end_point'].y

                    #print(x_start, y_start)
                    #print(x_end, y_end)


                    if Point(x_start, y_start).within(gadm['geometry']):
                        start_district = gadm['NAME_2']
                    if Point(x_end, y_end).within(gadm['geometry']):
                        end_district = gadm['NAME_2']
                except:
                    pass


                data.append({'geometry': traffic['Geometry'],
                            'TMC_PC': traffic['TMC_PC'],
                            'TMC_DE': traffic['TMC_DE'],
                             'TMC_QD': traffic['TMC_QD'],
                             'TMC_LE': traffic['TMC_LE'],
                            'FC': traffic['FC'],
                             'CF_SP': traffic['CF_SP'],
                             'CF_SU': traffic['CF_SU'],
                             'CF_FF': traffic['CF_FF'],
                             'CF_JF': traffic['CF_JF'],
                             'CF_CN': traffic['CF_CN'],
                             'PBT': traffic['PBT'],
                             'DE': traffic['DE'],
                             'endpoints': traffic['endpoints'],
                             'start_point': traffic['start_point'],
                             'end_point': traffic['end_point'],
                             'start_district': start_district,
                             'end_district': end_district
                            })

    final_df_startpoint = gpd.GeoDataFrame(data,columns=['geometry', 'TMC_PC', 'TMC_DE','TMC_QD', 'TMC_LE', 'FC', 
                                        'CF_SP', 'CF_SU', 'CF_FF', 'CF_JF', 'CF_CN', 'PBT', 'DE', 'endpoints', 'start_point', 'end_point', 'start_district', 'end_district'])
    
    final_df_startpoint.to_csv('traffic_flow_csv/' + filename + '.csv', index=False)
    


In [2]:
command = "elasticdump --input=http://10.4.130.10:9200/traffic_flow --output=D:/Datasets/MIMOS/traffic_flow_09092020_0800_0900.json --type=data --ignore-errors=true --size=-1 --searchBody '{\"query\": {\"range\":{\"PBT\":{\"gte\":\"2020-09-09T08:00:00\",\"lte\":\"2020-09-09T08:59:59\"}}}}'"
!elasticdump --input=http://10.4.130.10:9200/traffic_flow --output=D:/Datasets/MIMOS/traffic_flow_09092020_0800_0900.json --type=data --ignore-errors=true --size=20 --searchBody "{\"query\": {\"range\":{\"PBT\":{\"gte\":\"2020-09-09T08:00:00\",\"lte\":\"2020-09-09T08:59:59\"}}}}"

/bin/sh: 1: elasticdump: not found


In [14]:
pwd

'/blazing/Welcome_to_BlazingSQL_Notebooks/mag'

In [14]:
!df

Filesystem      1K-blocks       Used  Available Use% Mounted on
overlay        1845323628 1326025500  425538028  76% /
tmpfs               65536          0      65536   0% /dev
tmpfs           132028632          0  132028632   0% /sys/fs/cgroup
shm                 65536          0      65536   0% /dev/shm
/dev/sda2      1845323628 1326025500  425538028  76% /etc/hosts
/dev/md0       5581273824 1777440432 3522530560  34% /blazing/Welcome_to_BlazingSQL_Notebooks/mag
tmpfs           132028632         12  132028620   1% /proc/driver/nvidia
tmpfs            26405728       4328   26401400   1% /run/nvidia-persistenced/socket
udev            131990856          0  131990856   0% /dev/nvidia1
tmpfs           132028632          0  132028632   0% /proc/asound
tmpfs           132028632          0  132028632   0% /proc/acpi
tmpfs           132028632          0  132028632   0% /proc/scsi
tmpfs           132028632          0  132028632   0% /sys/firmware


In [ ]:
## For November
for i in range(30,31):
    for j in range(7,20):
        print("{:02d}".format(j))
        print('traffic_flow_' + "{:02d}".format(i) + '112020_' + "{:02d}".format(j) + '00_' + "{:02d}".format(j) + '59.json')
        filename = 'traffic_flow_' + "{:02d}".format(i) + '112020_' + "{:02d}".format(j) + '00_' + "{:02d}".format(j) + '59'
        generate_csv(filename)

## For December

for i in range(1,15):
    for j in range(7,20):
        print("{:02d}".format(j))
        print('traffic_flow_' + "{:02d}".format(i) + '122020_' + "{:02d}".format(j) + '00_' + "{:02d}".format(j) + '59.json')
        filename = 'traffic_flow_' + "{:02d}".format(i) + '122020_' + "{:02d}".format(j) + '00_' + "{:02d}".format(j) + '59'
        filename_json = filename + '.json'
        filename_csv = filename + '.csv'
        generate_csv(filename)
        

07
traffic_flow_30112020_0700_0759.json
08
traffic_flow_30112020_0800_0859.json
09
traffic_flow_30112020_0900_0959.json


In [21]:
main_arr = []
with open('raw_kibana/traffic_flow_30112020_0700_0759.json') as f:
    here_traffic_flow = f.readlines()
    for line in here_traffic_flow:
        line = json.loads(line)
        line = line['_source']
        #print(line)
        for flow_item_s in line['FIS']:
            for flow_item in flow_item_s['FI']:
                #print(flow_item)
                temp_arr = []
                for tmc_item in flow_item['TMC']:
                    #print(flow_item['TMC'][tmc_item])
                    temp_arr.append(flow_item['TMC'][tmc_item])

                ## To get SHP
                #shp_values = flow_item['SHP'][0]['value'][0]

                arr_lon = []
                arr_lat = []

                fc_temp = flow_item['SHP'][0]['FC']

                for shp in flow_item['SHP']:
                    shp_values = shp['value'][0]
                    coordinates_arr = shp_values.split(' ')
                    #print(coordinates_arr)

                    for item in coordinates_arr:
                        if len(item) > 0:
                            item = item.split(',')
                            arr_lat.append(float(item[0]))
                            arr_lon.append(float(item[1]))

                    #if fc_temp != shp['FC']:
                        #print(fc_temp, shp['FC'])
                        #print(line['PBT'])
                        #print(line['DE'])
                        #print('FC Diff')

                polygon_geom = LineString(zip(arr_lon, arr_lat))
                #temp_arr.append(flow_item['SHP'][0]['value'])
                temp_arr.append(polygon_geom)

                temp_arr.append(flow_item['SHP'][0]['FC'])


                ## To get all SHP
                """
                for cf_item in flow_item['SHP'][0]:   
                    temp_arr.append(flow_item['SHP'][0][cf_item])


                for cf_item in flow_item['CF'][0]:   
                    temp_arr.append(flow_item['CF'][0][cf_item])
                """
                try:
                    temp_arr.append(flow_item['CF'][0]['SP'])
                    temp_arr.append(flow_item['CF'][0]['SU'])
                    temp_arr.append(flow_item['CF'][0]['FF'])
                    temp_arr.append(flow_item['CF'][0]['JF'])
                    temp_arr.append(flow_item['CF'][0]['CN'])

                    temp_arr.append(line['PBT'])
                    temp_arr.append(line['DE'])



                    #print(len(temp_arr))
                    if len(temp_arr) == 13:
                        main_arr.append(temp_arr)
                except:
                    pass

In [46]:
here_traffic_flow_df = pd.DataFrame(main_arr)
here_traffic_flow_df.columns = ['TMC_PC', 'TMC_DE', 'TMC_QD', 'TMC_LE',
                        'Geometry', 'FC',
                         'CF_SP', 'CF_SU', 'CF_FF', 'CF_JF', 'CF_CN',
                           'PBT', 'DE']
here_traffic_flow_gdf = gpd.GeoDataFrame(here_traffic_flow_df, geometry='Geometry')
here_traffic_flow_gdf['endpoints'] = here_traffic_flow_gdf['Geometry'].apply(lambda x: x.boundary)
here_traffic_flow_gdf['start_point'] = here_traffic_flow_gdf['Geometry'].apply(lambda x: return_start_point(x))
here_traffic_flow_gdf['end_point'] = here_traffic_flow_gdf['Geometry'].apply(lambda x: return_end_point(x))

gadm = gpd.read_file('gadm_data/gadm36_MYS_2.shp')
kl_selangor_gadm_mys2 = gadm[gadm['NAME_1'].isin(['Kuala Lumpur', 'Selangor'])]
from shapely.geometry import Point, Polygon

data = []
for index2, traffic in here_traffic_flow_gdf.iterrows():
    for index, gadm in kl_selangor_gadm_mys2.iterrows():
        if gadm['geometry'].intersects(traffic['Geometry']):
            start_district = None
            end_district = None
            try:
                x_start = traffic['start_point'].x
                y_start = traffic['start_point'].y

                x_end = traffic['end_point'].x
                y_end = traffic['end_point'].y

                #print(x_start, y_start)
                #print(x_end, y_end)


                if Point(x_start, y_start).within(gadm['geometry']):
                    start_district = gadm['NAME_2']
                if Point(x_end, y_end).within(gadm['geometry']):
                    end_district = gadm['NAME_2']
            except:
                pass


            data.append({'geometry': traffic['Geometry'],
                        'TMC_PC': traffic['TMC_PC'],
                        'TMC_DE': traffic['TMC_DE'],
                         'TMC_QD': traffic['TMC_QD'],
                         'TMC_LE': traffic['TMC_LE'],
                        'FC': traffic['FC'],
                         'CF_SP': traffic['CF_SP'],
                         'CF_SU': traffic['CF_SU'],
                         'CF_FF': traffic['CF_FF'],
                         'CF_JF': traffic['CF_JF'],
                         'CF_CN': traffic['CF_CN'],
                         'PBT': traffic['PBT'],
                         'DE': traffic['DE'],
                         'endpoints': traffic['endpoints'],
                         'start_point': traffic['start_point'],
                         'end_point': traffic['end_point'],
                         'start_district': start_district,
                         'end_district': end_district
                        })

final_df_startpoint = gpd.GeoDataFrame(data,columns=['geometry', 'TMC_PC', 'TMC_DE','TMC_QD', 'TMC_LE', 'FC', 
                                    'CF_SP', 'CF_SU', 'CF_FF', 'CF_JF', 'CF_CN', 'PBT', 'DE', 'endpoints', 'start_point', 'end_point', 'start_district', 'end_district'])

In [49]:
print(len(final_df_startpoint))

125040


In [50]:
final_df_startpoint.head()

,geometry,TMC_PC,TMC_DE,TMC_QD,TMC_LE,FC,CF_SP,CF_SU,CF_FF,CF_JF,CF_CN,PBT,DE,endpoints,start_point,end_point,start_district,end_district
0,"LINESTRING (101.60047 3.17779, 101.60028 3.178...",6197,E1/LEBUHRAYA BARU LEMBAH KLANG,-,0.68890,1,60.00,60.00,60.0,0.00000,0.70,2020-11-30T07:59:41Z,,"(POINT (101.60047 3.17779), POINT (101.59679 3...",POINT (101.60047 3.17779),POINT (101.59679 3.1828),Petaling,Petaling
1,"LINESTRING (101.55429 3.09335, 101.55321 3.093...",6195,E1/LEBUHRAYA BARU LEMBAH KLANG (WESTBOUND),-,0.85224,1,49.00,49.00,50.0,0.20000,0.70,2020-11-30T07:59:41Z,,"(POINT (101.55429 3.09335), POINT (101.55011 3...",POINT (101.55429 3.09335),POINT (101.55011 3.08861),Petaling,Petaling
2,"LINESTRING (101.60047 3.17779, 101.60037 3.178...",6199,E1/LEBUHRAYA BARU LEMBAH KLANG,-,0.75774,1,60.09,60.09,60.0,0.00000,0.71,2020-11-30T07:59:41Z,,"(POINT (101.60047 3.17779), POINT (101.60307 3...",POINT (101.60047 3.17779),POINT (101.60307 3.18237),Petaling,Petaling
3,"LINESTRING (101.59472 3.18464, 101.59533 3.184...",6201,E1/LEBUHRAYA UTARA-SELATAN (EASTBOUND),-,1.05423,1,60.00,60.00,60.0,0.00000,0.70,2020-11-30T07:59:41Z,,"(POINT (101.59472 3.18464), POINT (101.60076 3...",POINT (101.59472 3.18464),POINT (101.60076 3.1781),Petaling,Petaling
4,"LINESTRING (101.60260 3.18206, 101.60244 3.181...",6203,E1/LEBUHRAYA UTARA-SELATAN (WESTBOUND),-,0.51601,1,59.00,59.00,60.0,0.12121,0.70,2020-11-30T07:59:41Z,,"(POINT (101.6026 3.18206), POINT (101.60076 3....",POINT (101.6026 3.18206),POINT (101.60076 3.1781),Petaling,Petaling


In [52]:
final_df_startpoint['start_district'].value_counts()

Kuala Lumpur      34704
Petaling          26340
Klang             17856
Hulu Langat       12960
Gombak             7944
Kuala Selangor     7152
Sepang             3984
Kuala Langat       3312
Hulu Selangor      2856
Sabak Bernam       2760
Name: start_district, dtype: int64

In [51]:
final_df_startpoint['start_district'].unique()

array(['Petaling', 'Kuala Lumpur', 'Hulu Langat', 'Sepang', None,
       'Gombak', 'Kuala Selangor', 'Hulu Selangor', 'Kuala Langat',
       'Klang', 'Sabak Bernam'], dtype=object)

In [37]:
traffic['start_point'].split(' ')[1].replace('(', '')

AttributeError: 'Point' object has no attribute 'split'

In [42]:
print(traffic['start_point'].x)

101.60047


In [59]:
!tar -czf archive.tar.gz traffic_flow_csv